In [4]:
import cv2                 #import openCV
import mediapipe as mp     #import mediapipe
import numpy as np         #import numpy
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
def calculate_angle(a,b,c):       #function to calculate angle
    a = np.array(a) #first point
    b = np.array(b) #mid point
    c = np.array(c) #end point
    
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle = 360-angle
    return angle

In [6]:
cap = cv2.VideoCapture('KneeBendVideo.mp4')      #render video; replace argument to '0' for cam

#variables required
counter = 0  #reps
pos = None   #current position
t=0          #timer

#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #recolour image as mediapipe takes RGB format and openCV gives BGR
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #make detection
        results = pose.process(image)
        
        #recolouring it back to BGR for openCV
        image.flags.writeable = True
        image = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        
        #extract landmarks(joints)
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get coordinates
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            #calculate angle
            angle = calculate_angle(left_hip,left_knee,left_ankle)
            
            #Visualize
            cv2.putText(image,str(int(angle)),
                       tuple(np.multiply(left_knee,[640,480]).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                       )
           
        #counter logic

            if angle>=160 and t==0:
                pos = "relaxed"  
            elif angle>140 and angle<160 and t!=0:
                pos = "keep bent"
                t=0
            if angle<140 and (pos=="relaxed" or pos=="keep bent" or pos==None):
                pos = None
                t=t+1
                cv2.putText(image, f'{str(int(t/10))}', (70, 215), cv2.FONT_ITALIC, 2, (0, 255, 255), 2)
        
                #counter = counter+1 
                if t%81==0:
                    counter = counter+1
                    pos =  "successful rep"
                    t=0         
        except:
            pass
        
        #render reps & position
        cv2.rectangle(image, (10,70), (280,150), (255,153,40),-1)
        
        #rep data
        cv2.putText(image,'REPS:',(15,100),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA
                       )
        cv2.putText(image,str(counter),(80,102),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA
                       )
        
        
        #position data
        cv2.putText(image,'POS:',(15,130),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA
                       )
        cv2.putText(image,pos,(80,132),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA
                       )
        
        
        
        #render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                 )
        
        
        cv2.imshow('Rootally AI Task', image)
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
